### **If you liked it, please upvote. It helps me a lot!!**

## XBoost for timeseries as simple as possible

After a simple Exploratory Data Analysis [EDA](https://www.kaggle.com/thalesgaluchi/march-2022-playground-start-exploring), the XGBoost cames to get a good result.

In this notebook, I am going to setup and run a XGBoost. It is as simple as possible. Withoun any further feature Enginieering.

In [ ]:
import numpy as np
import pandas as pd
import datetime

import xgboost as xgb
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')


In [ ]:
PATH ='../input/tabular-playground-series-mar-2022/'

train = pd.read_csv(PATH + 'train.csv', index_col = 'row_id', parse_dates=['time'])
test = pd.read_csv(PATH + 'test.csv', index_col = 'row_id', parse_dates=['time'])

# Let's keep those line below. Just in case we need split day and time in the future.
train['day'] = train.time.dt.date
train['time_day'] = train.time.dt.time

test['day'] = test.time.dt.date
test['time_day'] = test.time.dt.time

test.head()

## Prepare train dataset

Load the train dataset, convert time to day, month, hour, minutes. I removed the year because it is the same and a number too big in relation to the others.

The train was splited in 10% for validation. As it is a time series, I took the last 10% for validation (more recent data).

The previous model used all the available data. (0.528 in public leaderboard).
* **What if we use only Mondays for predictions?**

In [ ]:
data = pd.read_csv(PATH + 'train.csv', index_col = 'row_id', parse_dates=['time'])

data['day'] = data['time'].dt.day
data['month'] = data['time'].dt.month
# data['year'] = data['time'].dt.year
data['hour'] = data['time'].dt.hour
data['min'] = data['time'].dt.minute
data['dayofweek'] = data['time'].dt.dayofweek

directions = pd.get_dummies(data.direction)
data = data.join(directions)
 
# get number of rows for train and validation
rows = len(data)
train_rows = int(0.9*rows)
valid_rows = rows-train_rows

# Select just Mondays for the model. 
# This is the only difference from the previous model.
data = data[data.dayofweek == 0]

X_train, X_valid = data.iloc[:train_rows],  data.iloc[-valid_rows:] 

# X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
# X_valid, y_valid = valid.iloc[:,:-1], valid.iloc[:,-1]

y_train = X_train.loc[:,'congestion']
y_valid = X_valid.loc[:,'congestion']

cols_drop = ['time','direction', 'congestion']
# X_train = train.copy()
# X_valid = valid.copy()

X_train.drop(cols_drop, 1, inplace=True);
X_valid.drop(cols_drop, 1, inplace=True);

# display.display(X_train)
# display.display(X_valid)

## Prepare test dataset

In [ ]:
test = pd.read_csv(PATH + 'test.csv', index_col = 'row_id', parse_dates=['time'])

test['day'] = test['time'].dt.day
test['month'] = test['time'].dt.month
# data['year'] = data['time'].dt.year
test['hour'] = test['time'].dt.hour
test['min'] = test['time'].dt.minute
test['dayofweek'] = test['time'].dt.dayofweek

directions = pd.get_dummies(test.direction)
test = test.join(directions)
    
cols_drop = ['time','direction']
test.drop(cols_drop, 1, inplace=True);

test.head()


## Model

Simple XGBOOST setup. It can be improved using the parameters. 

In [ ]:
# This model gives me 5.829.
model = XGBRegressor(objective='reg:squarederror')

model.fit(X_train, y_train,
           eval_set = [(X_train, y_train), (X_valid, y_valid)],
           early_stopping_rounds=50,
           verbose=False)

yhat = model.predict(test)
yhat

## Submission file

The preparation os a submission file requires attention to be accepted. Then, I usually make a simple predicition to check if the file is correctly prepared.


In [ ]:
# 3rd submission using XGBoost.
subm = pd.read_csv(PATH + 'test.csv' )
preds = pd.DataFrame(yhat) #, columns=['congestion'] )
subm['congestion'] = preds

final = subm[['row_id', 'congestion']]
final.set_index('row_id', inplace = True)

final.to_csv('submission.csv')

Please, leave a comment o improve this notebook.

### **If you liked, give a UPVOTE**